In [27]:
import pandas as pd
import numpy as np
import kaggle
import re

In [ ]:
kaggle.api.authenticate()

# https://www.kaggle.com/oscarfry/tvtime-shows?select=tvtimeshows.csv
kaggle.api.dataset_download_files('oscarfry/tvtime-shows', path='./data/tvtimeshows', unzip = True)

# https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney 
kaggle.api.dataset_download_files('ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney', path='./data/streamings', unzip = True)

# https://www.kaggle.com/unanimad/emmy-awards
kaggle.api.dataset_download_files('unanimad/emmy-awards', path='./data/awards', unzip = True)]

# https://www.kaggle.com/unanimad/golden-globe-awards
kaggle.api.dataset_download_files('unanimad/golden-globe-awards', path='./data/awards' , unzip = True)

In [2]:
df_tvtime = pd.read_csv('data/tvtimeshows/tvtimeshows.csv')
# df_tvtime.head()

df_streamings = pd.read_csv('data/streamings/tv_shows.csv')
# df_streamings.head()

df_award_emmy = pd.read_csv('data/awards/the_emmy_awards.csv')
# df_award_emmy.head()

df_award_goldenglobe = pd.read_csv('data/awards/golden_globe_awards.csv')
# df_award_goldenglobe.head()

In [69]:
df_tvstatus = pd.read_csv('status/metacritic.csv')
df_tvstatus.head()

,name,season,status,tv_network,ano
0,20/20,SEASON 36,Renewed 5/13,ABC,2013
1,America's Funniest Home Videos,SEASON 24,Renewed 5/9,ABC,2013
2,The Assets,NaN,Canceled 1/10 after 2 eps aired,ABC,2013
3,The Astronaut Wives Club,Starts tbd,Delayed to 2015 for creative reasons,ABC,2013
4,The Bachelor,SEASON 18,Renewed 5/9,ABC,2013


In [70]:
def networks_more_popular(df_tt, networks): 
    column_names = list(df_tt.columns)
    df_most_popular = pd.DataFrame(columns = column_names)
    for i in networks:
        df = df_tt[df_tt['tv_network'].str.contains(i, flags=re.IGNORECASE, regex=True, na=False)]
        df_most_popular = df_most_popular.append(df,ignore_index=True)
        df_most_popular['tv_network'] = np.where(df_most_popular['tv_network'].str.contains(i, flags=re.IGNORECASE, regex=True, na=False),i,df_most_popular['tv_network'] )
    return df_most_popular

In [71]:
popular_networks = [ "ABC", "CBC", "NBC", "NETFLIX", "AMAZON", "DISNEY", "HULU", "HBO", "CRUNCHYROLL", "GLOBO", "APPLE", "FOX", "MTV",'FX','CBS','SYFY','THE CW','FREEF','TNT','IFC','STRZ','TVL','SHOWTIME','AMC','USA']
df_tvstatus_popular = networks_more_popular(df_tvstatus,popular_networks)

In [72]:
def standardize_status(df): 
    column_names = list(df.columns)
    column_names.append('changed_network')
    df_clean = pd.DataFrame(columns = column_names)
    df_clean = df
    df_clean['changed_network'] = False

    #canceled
    df_clean['status'] =  np.where(df_clean['status'].str.contains('CANCELED', flags=re.IGNORECASE, regex=True, na=False),'CANCELED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('order rescinded', flags=re.IGNORECASE, regex=True, na=False),'CANCELED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Likely to end', flags=re.IGNORECASE, regex=True, na=False),'CANCELED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('won\'t air', flags=re.IGNORECASE, regex=True, na=False),'CANCELED',df_clean['status'] )

    #renewed
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Renew' , flags=re.IGNORECASE, regex=True, na=False),'RENEWED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('order' , flags=re.IGNORECASE, regex=True, na=False),'RENEWED',df_clean['status'] )

    #renewed but changed network
    df_clean.loc[df_clean['status'].str.contains('mov' , flags=re.IGNORECASE, regex=True, na=False),'changed_network'] = True
    df_clean['status'] =  np.where(df_clean['status'].str.contains('mov' , flags=re.IGNORECASE, regex=True, na=False),'RENEWED',df_clean['status'] )  

    #delayed
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Delayed' , flags=re.IGNORECASE, regex=True, na=False),'DELAYED',df_clean['status'] )

    #end
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Ending' , flags=re.IGNORECASE, regex=True, na=False),'END',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('ended' , flags=re.IGNORECASE, regex=True, na=False),'END',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Final season' , flags=re.IGNORECASE, regex=True, na=False),'END',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('miniseries' , flags=re.IGNORECASE, regex=True, na=False),'END',df_clean['status'] )

   # Não tem Info
    nationality = ['Brazilian' , 'Spanish', 'German', 'Korean', 'Japanese', 'Swedish', 'French', 'Argentinian', 'Austalian', 'British', 'Belgian', 'Polish', 'Chinese', 'Italian', 'Mexican', 'Indian', 'Colombian', 'Thai', 'Taiwanese', 'Turkish', 'Norwegian', 'Lebanese', 'Israeli', 'Arabic', 'Dutch', 'Argentine','UK','Irish','USA']

    for i in nationality:
       df_clean['status'] =  np.where(df_clean['status'].str.contains(i , flags=re.IGNORECASE, regex=True, na=False),'NAN',df_clean['status'] ) 

    return(df_clean)


In [73]:
df_tvstatus_popular_cleaning = standardize_status(df_tvstatus_popular)
df_tvstatus_popular_cleaning.head()

,name,season,status,tv_network,ano,changed_network
0,20/20,SEASON 36,RENEWED,ABC,2013,False
1,America's Funniest Home Videos,SEASON 24,RENEWED,ABC,2013,False
2,The Assets,NaN,CANCELED,ABC,2013,False
3,The Astronaut Wives Club,Starts tbd,DELAYED,ABC,2013,False
4,The Bachelor,SEASON 18,RENEWED,ABC,2013,False


In [74]:
# df_tvstatus_popular_cleaning['tv_network'].unique()

In [75]:
df_tvstatus_popular_cleaning['status'].unique()

array(['RENEWED', 'CANCELED', 'DELAYED', nan,
       '1st season in ABC primetime',
       'Season cut to 10 episodes on 10/23', 'END',
       'New version of classic game show', 'Postponed to summer',
       'Pulled from schedule on 3/29 after 5 episodes',
       'Pulled from schedule 1/4 (4 eps unaired)',
       'Banished to Saturdays effective 7/8',
       "Sneak preview aired in March;  returns later in '18",
       'Fate uncertain; not included in 2014-15 schedule',
       'Pulled from schedule on 10/18',
       'Pulled from schedule 2/5; 7 eps unaired',
       'Not on schedule; likely will never air', 'Variety show', 'NAN',
       '2nd season already in production',
       'Continuation of 2000-07 WB/CW series',
       'UK series formerly known as Scrotal Recall', 'Canadian series',
       'Anime', 'Philippine series',
       'Two-part season (part 2 date is tbd)', 'Documentary series',
       'Both seasons will stream at once', 'Australian series',
       "Revival of classic chi

In [76]:
df_tvstatus_popular_cleaning[df_tvstatus_popular_cleaning['name'].str.contains('anne with an e', flags=re.IGNORECASE, regex=True, na=False)]

,name,season,status,tv_network,ano,changed_network
649,Anne With an E,NaN,RENEWED,NETFLIX,2016,False
724,Anne With an E,SEASON 2 starts tbd 2018,NaN,NETFLIX,2017,False
898,Anne With an E,SEASON 2,RENEWED,NETFLIX,2018,False
1168,Anne With an E,SEASON 3,CANCELED,NETFLIX,2019,False


In [77]:
df_tvstatus[df_tvstatus['status'].str.contains('order', flags=re.IGNORECASE, regex=True, na=False)]

,name,season,status,tv_network,ano
331,The Spoils of Babylon,NaN,Sequel ordered 7/7; will air in summer 2015,IFC,2013
576,Blunt Talk,Starts 8/22,2nd season already ordered,STRZ,2014
633,Fear the Walking Dead,tbd August,Two seasons ordered on 3/9/15,AMC,2014
1060,Blunt Talk,NaN,2nd season already ordered,STRZ,2015
1065,Brothers in Atlanta,starts tbd,Series order rescinded on 1/22,HBO,2015
1493,Vice Principals,NaN,Order is for 2 seasons; will end after that,HBO,2016
1514,Black Mirror,SEASON 3,Two seasons (S3 & S4) ordered by Netflix,Netflix,2016
1565,Skylanders Academy,NaN,Original order is for 2 seasons,Netflix,2016
1584,The Grand Tour,NaN,Initial order is for 3 seasons,Amazon Prime,2016
1599,Chance,NaN,Initial order is for 2 seasons,Hulu,2016


In [78]:
df_tvstatus_popular_cleaning[df_tvstatus_popular_cleaning['changed_network'] == True]

,name,season,status,tv_network,ano,changed_network
257,The Alec Baldwin Show,NaN,RENEWED,ABC,2018,True
582,The Gilded Age,starts tbd,RENEWED,NBC,2018,True
684,The Last Kingdom,SEASON 2,RENEWED,NETFLIX,2016,True
911,Black Spot,SEASON 2,RENEWED,NETFLIX,2018,True
1453,Fortitude,SEASON 2,RENEWED,AMAZON,2016,True
1621,There's ... Johnny!,NaN,RENEWED,HULU,2017,True
1642,UnREAL,SEASON 4,RENEWED,HULU,2018,True
1655,High Fidelity,NaN,RENEWED,HULU,2019,True
1676,Veronica Mars,SEASON 4,RENEWED,HULU,2019,True
1878,American Dad!,SEASON 9/10,RENEWED,FOX,2013,True


In [84]:
df_tvstatus[df_tvstatus['name'].str.contains('UnREA', flags=re.IGNORECASE, regex=True, na=False)]

,name,season,status,tv_network,ano
819,UnReal,NaN,Renewed for a 2nd season on 7/6,LIFE,2014
1803,UnReal,SEASON 2,Renewed for a 3rd season on 6/2,LIFE,2016
2500,UnReal,SEASON 3,Renewed for a 4th season on 7/28,LIFE,2017
3082,UnREAL,SEASON 4,Final season; moves to Hulu from Lifetime,Hulu,2018


In [85]:
df_tvstatus_popular_cleaning[df_tvstatus_popular_cleaning['name'].str.contains('UnREA', flags=re.IGNORECASE, regex=True, na=False)]

,name,season,status,tv_network,ano,changed_network
1642,UnREAL,SEASON 4,RENEWED,HULU,2018,True
